In [1]:
import pandas as pd
import yfinance as yf

In [14]:
mcd = yf.Ticker("mcd")
mcd_history = mcd.history(period="max")

# Create rolling weekly average column
mcd_history["weekly_avg_close"] = mcd_history["Close"].rolling(7).mean()

# Create weekly average close percent column
mcd_history["weekly_avg_close_percent"] = (mcd_history["Close"] - mcd_history["weekly_avg_close"]) / mcd_history["Close"] * 100

# Create rolling monthly average column
mcd_history["monthly_avg_close"] = mcd_history["Close"].rolling(30).mean()

# Create monthly average close percent column
mcd_history["monthly_avg_close_percent"] = (mcd_history["Close"] - mcd_history["monthly_avg_close"]) / mcd_history["Close"] * 100

# Create rolling semi_annual average column
mcd_history["semi_annual_avg_close"] = mcd_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
mcd_history["semi_annual_avg_close_percent"] = (mcd_history["Close"] - mcd_history["semi_annual_avg_close"]) / mcd_history["Close"] * 100

# Create rolling annual average column
mcd_history["annual_avg_close"] = mcd_history["Close"].rolling(365).mean()

# Create annual average close percent column
mcd_history["annual_avg_close_percent"] = (mcd_history["Close"] - mcd_history["annual_avg_close"]) / mcd_history["Close"] * 100


# Create rolling two_year average column
mcd_history["two_year_avg_close"] = mcd_history["Close"].rolling(730).mean()

# Create two_year average close percent column
mcd_history["two_year_avg_close_percent"] = (mcd_history["Close"] - mcd_history["two_year_avg_close"]) / mcd_history["Close"] * 100

# Drop NAs
mcd_history = mcd_history.dropna()

mcd_price_action = mcd_history[["Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
mcd_price_action = mcd_price_action.reset_index()
mcd_price_action

,Date,Close,weekly_avg_close_percent,monthly_avg_close_percent,semi_annual_avg_close_percent,annual_avg_close_percent,two_year_avg_close_percent,weekly_avg_close,monthly_avg_close,semi_annual_avg_close,annual_avg_close,two_year_avg_close
0,2022-11-11,271.390015,-1.458625,5.654966,8.496746,9.779858,18.530582,275.348576,256.043000,248.330693,244.848456,221.099866
1,2022-11-10,275.880005,0.245449,7.681602,10.031204,11.293144,19.899466,275.202859,254.688000,248.205919,244.724478,220.981356
2,2022-11-09,277.790009,1.082012,8.814095,10.707615,11.950849,20.495752,274.784289,253.305333,248.045323,244.591744,220.854858
3,2022-11-08,278.399994,1.561984,9.502992,10.964056,12.193842,20.715915,274.051431,251.943666,247.876063,244.452337,220.726887
4,2022-11-07,276.410004,1.053818,9.354342,10.381290,11.612102,20.191084,273.497144,250.553666,247.715079,244.312992,220.599828


In [15]:
pep = yf.Ticker("PEP")
pep_history = pep.history(period="max")

# Create rolling weekly average column
pep_history["weekly_avg_close"] = pep_history["Close"].rolling(7).mean()

# Create weekly average close percent column
pep_history["weekly_avg_close_percent"] = (pep_history["Close"] - pep_history["weekly_avg_close"]) / pep_history["Close"] * 100

# Create rolling monthly average column
pep_history["monthly_avg_close"] = pep_history["Close"].rolling(30).mean()

# Create monthly average close percent column
pep_history["monthly_avg_close_percent"] = (pep_history["Close"] - pep_history["monthly_avg_close"]) / pep_history["Close"] * 100

# Create rolling semi_annual average column
pep_history["semi_annual_avg_close"] = pep_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
pep_history["semi_annual_avg_close_percent"] = (pep_history["Close"] - pep_history["semi_annual_avg_close"]) / pep_history["Close"] * 100

# Create rolling annual average column
pep_history["annual_avg_close"] = pep_history["Close"].rolling(365).mean()

# Create annual average close percent column
pep_history["annual_avg_close_percent"] = (pep_history["Close"] - pep_history["annual_avg_close"]) / pep_history["Close"] * 100

# Create rolling two_year average column
pep_history["two_year_avg_close"] = pep_history["Close"].rolling(730).mean()

# Create two_year average close percent column
pep_history["two_year_avg_close_percent"] = (pep_history["Close"] - pep_history["two_year_avg_close"]) / pep_history["Close"] * 100


# Drop NAs
pep_history = pep_history.dropna()

pep_price_action = pep_history[["Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
pep_price_action = pep_price_action.reset_index()
pep_price_action

,Date,Close,weekly_avg_close_percent,monthly_avg_close_percent,semi_annual_avg_close_percent,annual_avg_close_percent,two_year_avg_close_percent,weekly_avg_close,monthly_avg_close,semi_annual_avg_close,annual_avg_close,two_year_avg_close
0,2022-11-11,178.050003,-0.756609,2.283442,5.338830,8.846193,18.002854,179.397145,173.984334,168.544216,162.299356,145.995922
1,2022-11-10,181.699997,1.252454,4.517701,7.279675,10.730490,19.689054,179.424288,173.491334,168.472828,162.202696,145.924986
2,2022-11-09,178.550003,-0.414449,3.114908,5.696968,9.216106,18.315086,179.290002,172.988334,168.378067,162.094646,145.848416
3,2022-11-08,180.710007,0.546261,4.455208,6.869918,10.357109,19.331299,179.722859,172.659001,168.295378,161.993675,145.776415
4,2022-11-07,180.210007,0.149829,4.462020,6.665296,10.167252,19.148450,179.940000,172.169000,168.198476,161.887600,145.702583


In [10]:
msft = yf.Ticker("MSFT")
msft_history = msft.history(period="max")

# Create rolling weekly average column
msft_history["weekly_avg_close"] = msft_history["Close"].rolling(7).mean()

# Create weekly average close percent column
msft_history["weekly_avg_close_percent"] = (msft_history["Close"] - msft_history["weekly_avg_close"]) / msft_history["Close"] * 100

# Create rolling monthly average column
msft_history["monthly_avg_close"] = msft_history["Close"].rolling(30).mean()

# Create monthly average close percent column
msft_history["monthly_avg_close_percent"] = (msft_history["Close"] - msft_history["monthly_avg_close"]) / msft_history["Close"] * 100

# Create rolling semi_annual average column
msft_history["semi_annual_avg_close"] = msft_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
msft_history["semi_annual_avg_close_percent"] = (msft_history["Close"] - msft_history["semi_annual_avg_close"]) / msft_history["Close"] * 100

# Create rolling annual average column
msft_history["annual_avg_close"] = msft_history["Close"].rolling(365).mean()

# Create annual average close percent column
msft_history["annual_avg_close_percent"] = (msft_history["Close"] - msft_history["annual_avg_close"]) / msft_history["Close"] * 100

# Create rolling two_year average column
msft_history["two_year_avg_close"] = msft_history["Close"].rolling(730).mean()

# Create two_year average close percent column
msft_history["two_year_avg_close_percent"] = (msft_history["Close"] - msft_history["two_year_avg_close"]) / msft_history["Close"] * 100


# Drop NAs
msft_history = msft_history.dropna()

msft_price_action = msft_history[["Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
msft_price_action = msft_price_action.reset_index()
msft_price_action

,Date,Close,weekly_avg_close_percent,monthly_avg_close_percent,semi_annual_avg_close_percent,annual_avg_close_percent,weekly_avg_close,monthly_avg_close,semi_annual_avg_close,annual_avg_close
0,2022-11-11,247.110001,7.098633,5.129025,-7.520840,-14.403783,229.568569,234.435666,265.694748,282.703188
1,2022-11-10,242.979996,7.107580,3.711415,-9.436863,-16.343934,225.709998,233.962000,265.909684,282.692485
2,2022-11-09,224.509995,0.407871,-4.128697,-18.546318,-25.917517,223.594284,233.779333,266.148332,282.697410
3,2022-11-08,228.869995,1.829483,-2.386219,-16.443023,-23.543153,224.682857,234.331333,266.503141,282.753209
4,2022-11-07,227.869995,0.959818,-2.945834,-17.120218,-24.107276,225.682857,234.582667,266.881836,282.803245
